<a href="https://colab.research.google.com/github/SubramDas/Homomorphic-Encryption-Secure-KNN/blob/main/Paillier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install phe
import numpy as np
import time

total_encription_time = 0

In [ ]:
columns = 10
rows = 100000

D = np.random.uniform(low=1, high=100, size=(rows, columns))
D = np.round(D, 1)

q = np.random.uniform(low = 1, high = 100, size=columns)
q = np.round(q, 1)

In [ ]:
#Key Generation

def generate_key(d, c, epsilon):

  #computing eta
  eta = d + c + epsilon + 1

  # Generate an base secret matrix Mbase of eta, eta
  M = np.random.uniform(low=1, high=100, size=(eta, eta))
  M = np.round(M, 0)

  # M_inv = np.linalg.inv(M)

  # Generate a (d + 1) dimensional long term secret vector S
  S = np.random.uniform(low=1, high=100, size=(d + 1))
  S = np.round(S, 0)

  # S = [12, 5, 30]

  # Generate a c-dimensional vector (τ/w)
  tau = np.random.uniform(low=1, high=100, size=c)
  tau = np.round(tau, 0)
  # tau = [21]

  # Generate a permutation function π of η numbers
  pi = np.random.permutation(eta)
  pi = np.round(pi, 0)
  # pi = [3, 1, 4, 5, 2]

  # Create key

  key = {
      'S': S,
      'tau': tau,
      'pi': pi,
      'M': M
  }

  return key


In [ ]:
# Taking parameters

d = len(D[0])
c = 5
epsilon = 5

key = generate_key(d, c, epsilon)

# print(key)

In [ ]:
# D = the private database of DO consisting of m points
# D = {p1, p2, ... pm} where p1 = {pi1, pi2,... pid}
# ||pi|| = euclidean norm

# D = {p1, p2}

# tic = time.time()

# p1 = [10, 2.7, 5.3]
# p2 = [5.3, 8.6, 2.5]
# p3 = [1.3, 5.6, 8.5]
# q = [2.5, 3.1, 1.6]


# database D of n vectors with d dimensions ( n=1, d=2)
# (Data Owner DO )
# D = [p1, p2, p3]

# D = np.random.uniform(low = 1, high = 10, size=(2,2))

num_rows = len(D)

random_vectors=[]

for i in range(num_rows):
  epsilon_dim = np.random.uniform(low = 1, high = 100, size=epsilon)
  epsilon_dim = np.round(epsilon_dim, 0)
  random_vectors.append(epsilon_dim)

def DBEnc(D, key):

  D_dash = []

  M = key['M']
  M_inv = np.linalg.inv(M)
  tau = key['tau']
  S = key['S']


  S_size = len(S) - 1
  # print(S_size)

  # Collect S from 1 to d
  for i in range(len(D)):

    temp_pi_dash = []
    l2_norm_sq = 0

    # S-(d+1)
    for j in range(S_size):
      temp_pi_dash.append(S[j] - 2*D[i][j])

      l2_norm_sq += D[i][j]*D[i][j]

    temp_pi_dash.append(S[-1] + l2_norm_sq)

    for j in range(len(tau)):
      temp_pi_dash.append(tau[j])


    for k in range(epsilon):
      temp_pi_dash.append(random_vectors[i][k])

    # print(temp_pi_dash)
    permutation_order = key['pi']
    permuted_array = [temp_pi_dash[i - 1] for i in permutation_order]

    # print("permutaiton", permuted_array)
    # print("M_inv", M_inv)


    pi_dash = np.dot(permuted_array, M_inv)

    # print(pi_dash)

    D_dash.append(pi_dash)

  return D_dash

D_dash = DBEnc(D, key)

# toc = time.time()

# total_encription_time += toc - tic




In [ ]:
# Query Encryption
import json
import phe
from phe import paillier

In [ ]:
# Step1
tic = time.time()

pubkey, privkey = paillier.generate_paillier_keypair(n_length=1024)
# print(pubkey, privkey)

toc = time.time()
total_encription_time += toc - tic

In [ ]:
tic = time.time()

encrypted_pailllier = []
for i in range(len(q)):
  temp = q[i]*10
  encrypted_pailllier.append(pubkey.encrypt(temp))

toc = time.time()
total_encription_time += toc - tic

In [ ]:
tic = time.time()

beta_q = np.random.uniform(low =1, high = 100, size = 1)
beta_q = np.round(beta_q, 0)

R_q =  np.random.uniform(low =1, high = 100, size = c)
R_q = np.round(R_q, 0)

toc = time.time()
total_encription_time += toc - tic

In [ ]:
# q_dot = pi(q, 1, R_q, O_epsilon) O_epsilon is epsilon dim zero vector
tic = time.time()

temp_array = []

size = len(q) + 1 + epsilon + c


for j in range(len(q)):
  temp_array.append(10*q[j])

temp_array.append(10)

for j in range(c):
  temp_array.append(R_q[j])

for j in range(epsilon):
  temp_array.append(0)

Encrypted_temp_array = []

for i in range(size):
  Encrypted_temp_array.append(pubkey.encrypt(temp_array[i]))

permutation_order = key['pi']

q_dot = [Encrypted_temp_array[i - 1] for i in permutation_order]

toc = time.time()
total_encription_time += toc - tic

In [ ]:
tic = time.time()
# Computing Aq

eta = d + c + epsilon + 1

M = key['M']

def compute_Aq(permutation_order, M, q_dot, beta_q, R_q):
  # print(np.shape(M))

  q_dot_transpose = np.transpose(q_dot)
  # print(beta_q*M_inv[0])
  A_q = []


  for i in range(eta):
    beta_q_M = 10 * beta_q * M[i]
    dot_product = np.dot(beta_q_M , q_dot_transpose)

    A_q.append(dot_product)

  return A_q

compute_Aq(permutation_order, M, q_dot, beta_q, R_q)

toc = time.time()
total_encription_time += toc - tic

In [ ]:
# QU decrypting
A_q = compute_Aq(permutation_order, M, q_dot, beta_q, R_q)
q_dash_decipher = []
for i in range(eta):
  decipher = privkey.decrypt(A_q[i])/100
  q_dash_decipher.append(decipher)

print(q_dash_decipher)

[295054.4, 362060.8, 321224.0, 434465.6, 281684.8, 405075.2, 373030.4, 411694.4, 357745.6, 255729.6, 498747.2, 409228.8, 359476.8, 436288.0, 235232.0, 251544.0, 492297.6, 353472.0, 334640.0, 515747.2, 365433.6]


In [ ]:
# KNN computation
# D_dash = DBEnc(D, key)
# print(D_dash)

# Paillier KNN
tic = time.time()
def KNNcom(D_dash, q_dash_decipher, K):
  distances = []
  for i in range(len(D_dash)):
    distances.append([np.dot(D_dash[i],q_dash_decipher), i])
  return distances
  # print("ddash", D_dash)
  # print("qdash" ,q_dash_decipher)

distances = KNNcom(D_dash, q_dash_decipher, 10)
print(np.shape(distances))

# Sort the array based on the values in the first column (axis=0)
sorted_array_2D = sorted(distances, key=lambda x: x[0])

for i in range(10):
  print(sorted_array_2D[i][1])
# sorted_indices = np.argsort(distances[:, 0])

# Use the sorted indices to rearrange the rows of the array
# sorted_array_2d = distances[sorted_indices]

# print(np.argmin(distances))
# print(sorted(distances)[:10])

toc = time.time()

paillierKNN_time = toc -tic
print(paillierKNN_time)

(100000, 2)
64814
47905
88148
23180
64976
90172
86862
2680
28613
32257
1.3108453750610352


In [ ]:
# q_dot_transpose = np.transpose(q_dot)
# print(np.shape(q_dash_decipher))

from scipy.spatial import distance
min_dis = []
for i in range(len(D)):
  min_dis.append(distance.euclidean(D[i], q))

# print(min_dis)

print(np.argmin(min_dis))

# print(distances)





64814


In [ ]:
# import time
# from google.colab import drive

# # Mount Google Drive
# drive.mount('/content/drive')


# # Record the start time
# start_time = time.time()

# # Run your notebook using the %run magic command
# %run "/content/drive/My Drive/Colab Notebooks/KnnComp.ipynb"

# # Record the end time
# end_time = time.time()

# # Calculate the elapsed time
# elapsed_time = end_time - start_time

# # Print the runtime
# print(f"Runtime: {elapsed_time} seconds")

#Plaintext
from sklearn.neighbors import NearestNeighbors

tic = time.time()
k = 10
query = q.reshape(1, -1)

nn_model = NearestNeighbors(n_neighbors=k)
nn_model.fit(D)

distances, indices = nn_model.kneighbors(query)

print(f"{k}-Nearest Neighbors Indices:", indices)

toc = time.time()

plaintext_time = toc-tic

print(plaintext_time)


10-Nearest Neighbors Indices: [[64814 47905 88148 23180 64976 90172 86862  2680 28613 32257]]
0.3786635398864746


In [ ]:
print(total_encription_time)

1.4487855434417725
